**Instituto Tecnológico de Costa Rica - TEC**

***Inteligencia Artificial***

*Docente: Kenneth Obando Rodríguez*

---
# Trabajo Corto 3: Árboles de Decisión
---
Estudiantes:
- Ariel Leyva c.2022175018

Link del Cuaderno (recuerde configurar el acceso a público):

    
- [Link de su respuesta](https://github.com/hart-venus/tc4-ia/)

    **Nota:** Este trabajo tiene como objetivo promover la comprensión de la materia y su importancia en la elección de algoritmos. Los alumnos deben evitar copiar y pegar directamente información de fuentes externas, y en su lugar, demostrar su propio análisis y comprensión.

### Entrega
Debe entregar un archivo comprimido por el TecDigital, incluyendo un documento pdf con los resultados de los experimentos y pruebas. La fecha de entrega es el miércoles 1 de mayo a las 10:00 p.m.

Instrucciones:

Las alternativas se rifarán en clase utilizando números aleatorios. Deberá realizar la asignación propuesta. Si realiza ambos ejercicios, recibirá 20 puntos en **la nota porcentual de la actividad**, para aplicar a la totalidad de los puntos extra es necesario que ambas actividades se completen al 100%


## Actividad - Taller

1. Cree una clase nodo con atributos necesarios para un árbol de decisión: feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha


In [8]:
import numpy as np
from sklearn.datasets import load_iris


class Nodo:
    def __init__(self, feature=None, umbral=None, gini=None, cantidad_muestras=None, valor=None, izquierda=None, derecha=None):
        self.feature = feature  # Índice de la característica (feature) para la división
        self.umbral = umbral  # Valor umbral para la división
        self.gini = gini  # Impureza de Gini en el nodo
        self.cantidad_muestras = cantidad_muestras  # Cantidad de muestras en el nodo
        self.valor = valor  # Valor de predicción (para nodos hoja)
        self.izquierda = izquierda  # Nodo hijo izquierdo
        self.derecha = derecha  # Nodo hijo derecho


Explicación de los atributos:

- feature: Representa el índice de la característica (feature) utilizada para la división en el nodo. Es None para los nodos hoja.
- umbral: Representa el valor umbral utilizado para la división en el nodo. Las muestras con valores menores o iguales al umbral se envían al nodo hijo izquierdo, mientras que las muestras con valores mayores se envían al nodo hijo derecho. Es None para los nodos hoja.
- gini: Representa la impureza de Gini en el nodo, que mide la calidad de la división. Un valor de Gini más bajo indica una división más pura.
- cantidad_muestras: Representa la cantidad de muestras que llegan al nodo durante el entrenamiento del árbol.
- valor: Representa el valor de predicción para los nodos hoja. En un árbol de clasificación, suele ser la clase mayoritaria entre las muestras que llegan al nodo hoja. En un árbol de regresión, puede ser el promedio de los valores de las muestras en el nodo hoja.
- izquierda: Representa el nodo hijo izquierdo, que contiene las muestras con valores menores o iguales al umbral de división. Es None para los nodos hoja.
- derecha: Representa el nodo hijo derecho, que contiene las muestras con valores mayores al umbral de división. Es None para los nodos hoja.


2. Crea una clase que implementa un árbol de decisión, utilice las funciones presentadas en clase, además incluya los siguientes hyperparámetros:
   - max_depth: Cantidad máxima de variables que se pueden explorar
   - min_split_samples: Cantidad mínima de muestras que deberá tener un nodo para poder ser dividido
   - criterio: función que se utilizará para calcular la impuridad.


In [9]:
class ArbolDecision:
    def __init__(self, max_depth=None, min_split_samples=2, criterio='gini'):
        self.max_depth = max_depth
        self.min_split_samples = min_split_samples
        self.criterio = criterio
        self.root = None

    def _calcular_gini(self, y):
        _, counts = np.unique(y, return_counts=True) # cuenta las ocurrencias de cada clase
        proporciones = counts / len(y) # calcula la proporción de cada clase
        gini = 1 - np.sum(proporciones ** 2) # calcula la impureza de Gini
        return gini 

    def _dividir(self, X, y, feature, umbral):
        izquierda_indices = X[:, feature] <= umbral
        derecha_indices = X[:, feature] > umbral # Divide basado en los umbrales
        X_izquierda, y_izquierda = X[izquierda_indices], y[izquierda_indices]
        X_derecha, y_derecha = X[derecha_indices], y[derecha_indices]
        return X_izquierda, y_izquierda, X_derecha, y_derecha

    def _construir_arbol(self, X, y, depth=0):
        cantidad_muestras = len(y)
        if cantidad_muestras < self.min_split_samples or depth == self.max_depth: # caso base
            valor = np.bincount(y).argmax()
            return Nodo(cantidad_muestras=cantidad_muestras, valor=valor)

        mejor_feature, mejor_umbral, mejor_gini = None, None, float('inf')
        for feature in range(X.shape[1]):
            umbrales = np.unique(X[:, feature])
            for umbral in umbrales: # construye el arbol por cada umbral
                X_izquierda, y_izquierda, X_derecha, y_derecha = self._dividir(X, y, feature, umbral)
                if len(y_izquierda) > 0 and len(y_derecha) > 0: # si hay muestras en ambos nodos hijos
                    gini_izquierda = self._calcular_gini(y_izquierda)
                    gini_derecha = self._calcular_gini(y_derecha)
                    gini = (len(y_izquierda) * gini_izquierda + len(y_derecha) * gini_derecha) / cantidad_muestras
                    if gini < mejor_gini:
                        mejor_feature, mejor_umbral, mejor_gini = feature, umbral, gini

        if mejor_feature is None: # si no se puede dividir, caso base
            valor = np.bincount(y).argmax()
            return Nodo(cantidad_muestras=cantidad_muestras, valor=valor)

        X_izquierda, y_izquierda, X_derecha, y_derecha = self._dividir(X, y, mejor_feature, mejor_umbral)
        nodo_izquierdo = self._construir_arbol(X_izquierda, y_izquierda, depth + 1) # crear hijos recursivamente
        nodo_derecho = self._construir_arbol(X_derecha, y_derecha, depth + 1)

        return Nodo(feature=mejor_feature, umbral=mejor_umbral, gini=mejor_gini,
                    cantidad_muestras=cantidad_muestras, izquierda=nodo_izquierdo, derecha=nodo_derecho)

    def fit(self, X, y): # metodo publico para entrenar el modelo
        self.root = self._construir_arbol(X, y)

    def _predecir(self, x, nodo): 
        if nodo.valor is not None:
            return nodo.valor
        if x[nodo.feature] <= nodo.umbral:
            return self._predecir(x, nodo.izquierda)
        else:
            return self._predecir(x, nodo.derecha)

    def predict(self, X): # metodo publico para hacer predicciones
        return np.array([self._predecir(x, self.root) for x in X])

3. Divida los datos en los conjuntos tradicionales de entrenamiento y prueba, de forma manual, sin utilizar las utilidades de sklearn (puede utilizar índices de Numpy o Pandas)

In [10]:
# Cargar datos de prueba (Iris dataset)
iris = load_iris()
X, y = iris.data, iris.target

# Dividir los datos en conjuntos de entrenamiento y prueba utilizando índices de NumPy
indices = np.arange(len(X))
np.random.shuffle(indices)
train_size = int(0.8 * len(X))
train_indices = indices[:train_size]
test_indices = indices[train_size:]
X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]


4. Implemente una función que se llame `validacion_cruzada` que entrene $k$ modelos y reporte las métricas obtenidas:
  a. Divida el conjunto de entrenamiento en $k$ subconjuntos excluyentes
  b. Para cada uno de los $k$ modelos, utilice un subconjunto como validación
  c. Reporte la media y la desviación estándar para cada una de las métricas, todo debe realizarse solo usando Numpy:
    - Accuracy
    - Precision
    - Recall
    - F1


In [11]:
def validacion_cruzada(modelo, X, y, k=5):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    fold_sizes = np.full(k, len(X) // k, dtype=int)
    fold_sizes[:len(X) % k] += 1
    current = 0
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        val_indices = indices[start:stop]
        train_indices = np.concatenate((indices[:start], indices[stop:]))
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_val)

        accuracy = np.mean(y_pred == y_val)
        precision = np.sum((y_pred == 1) & (y_val == 1)) / np.sum(y_pred == 1)
        recall = np.sum((y_pred == 1) & (y_val == 1)) / np.sum(y_val == 1)
        f1 = 2 * precision * recall / (precision + recall)

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        current = stop

    print("Accuracy - Media: {:.4f}, Desviación Estándar: {:.4f}".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
    print("Precision - Media: {:.4f}, Desviación Estándar: {:.4f}".format(np.mean(precision_scores), np.std(precision_scores)))
    print("Recall - Media: {:.4f}, Desviación Estándar: {:.4f}".format(np.mean(recall_scores), np.std(recall_scores)))
    print("F1 - Media: {:.4f}, Desviación Estándar: {:.4f}".format(np.mean(f1_scores), np.std(f1_scores)))


5. Entrene 10 combinaciones distintas de parámetros para su implementación de Arbol de Decisión y utilizando su implementación de `validacion_cruzada`.


In [12]:
# Definir las combinaciones de parámetros
param_combinations = [
    {'max_depth': 2, 'min_split_samples': 2},
    {'max_depth': 2, 'min_split_samples': 5},
    {'max_depth': 3, 'min_split_samples': 2},
    {'max_depth': 3, 'min_split_samples': 5},
    {'max_depth': 4, 'min_split_samples': 2},
    {'max_depth': 4, 'min_split_samples': 5},
    {'max_depth': 5, 'min_split_samples': 2},
    {'max_depth': 5, 'min_split_samples': 5},
    {'max_depth': 6, 'min_split_samples': 2},
    {'max_depth': 6, 'min_split_samples': 5}
]

# Entrenar y evaluar cada combinación de parámetros
for i, params in enumerate(param_combinations, 1):
    print(f"Combinación de parámetros {i}:")
    print(params)
    
    # Crear y entrenar el árbol de decisión con los parámetros actuales
    arbol = ArbolDecision(**params)
    
    # Realizar validación cruzada
    validacion_cruzada(arbol, X, y, k=5)
    
    print()


Combinación de parámetros 1:
{'max_depth': 2, 'min_split_samples': 2}
Accuracy - Media: 0.9133, Desviación Estándar: 0.0542
Precision - Media: 0.8316, Desviación Estándar: 0.1360
Recall - Media: 0.9033, Desviación Estándar: 0.0897
F1 - Media: 0.8580, Desviación Estándar: 0.0905

Combinación de parámetros 2:
{'max_depth': 2, 'min_split_samples': 5}
Accuracy - Media: 0.9267, Desviación Estándar: 0.0533
Precision - Media: 0.8839, Desviación Estándar: 0.0896
Recall - Media: 0.9035, Desviación Estándar: 0.1103
F1 - Media: 0.8895, Desviación Estándar: 0.0859

Combinación de parámetros 3:
{'max_depth': 3, 'min_split_samples': 2}
Accuracy - Media: 0.9600, Desviación Estándar: 0.0327
Precision - Media: 0.9333, Desviación Estándar: 0.0565
Recall - Media: 0.9402, Desviación Estándar: 0.0865
F1 - Media: 0.9347, Desviación Estándar: 0.0596

Combinación de parámetros 4:
{'max_depth': 3, 'min_split_samples': 5}
Accuracy - Media: 0.9600, Desviación Estándar: 0.0133
Precision - Media: 0.9470, Desviació

6. Utilizando los resultados obtenidos analice cuál y porqué es el mejor modelo para ser usado en producción.

Basándonos en los resultados obtenidos de las 10 combinaciones de parámetros evaluadas mediante validación cruzada, podemos analizar cuál es el mejor modelo para ser utilizado en producción.

Según los resultados, la combinación de parámetros número 4 parece ser la mejor opción. Los parámetros utilizados en esta combinación son:
- `max_depth`: 3
- `min_split_samples`: 5

Las métricas obtenidas para esta combinación son las siguientes:
- Accuracy - Media: 0.9600, Desviación Estándar: 0.0133
- Precision - Media: 0.9470, Desviación Estándar: 0.0720
- Recall - Media: 0.9386, Desviación Estándar: 0.0516
- F1 - Media: 0.9389, Desviación Estándar: 0.0211

Analizando estas métricas, podemos observar que:

1. La combinación número 4 logra una alta precisión (accuracy) con una media de 0.9600 y una baja desviación estándar de 0.0133. Esto indica que el modelo tiene un buen rendimiento general y es consistente en sus predicciones.

2. La precisión (precision) también es alta, con una media de 0.9470 y una desviación estándar relativamente baja de 0.0720. Esto sugiere que el modelo tiene una alta proporción de verdaderos positivos entre las instancias que predice como positivas.

3. El recall también es alto, con una media de 0.9386 y una desviación estándar de 0.0516. Esto indica que el modelo es capaz de identificar correctamente una gran proporción de las instancias positivas reales.

4. La puntuación F1, que es una media armónica de la precisión y el recall, también es alta con una media de 0.9389 y una desviación estándar baja de 0.0211. Esto sugiere un buen equilibrio entre la precisión y el recall.

Además, la combinación número 4 utiliza una profundidad máxima (`max_depth`) de 3 y un número mínimo de muestras para dividir (`min_split_samples`) de 5. Estos parámetros evitan que el árbol de decisión sea demasiado complejo y propenso al sobreajuste, al limitar la profundidad y requerir un número mínimo de muestras para realizar una división.

En comparación con las otras combinaciones de parámetros, la combinación número 4 muestra un buen equilibrio entre el rendimiento y la complejidad del modelo. Tiene métricas altas y consistentes, lo que sugiere que generalizará bien a nuevos datos.

Por lo tanto, basándonos en estos resultados y análisis, se recomienda utilizar el modelo entrenado con la combinación de parámetros número 4 para su uso en producción. Este modelo ha demostrado un buen rendimiento general, una alta precisión y recall, y un buen equilibrio entre la complejidad y la capacidad de generalización.


7. Compruebe las métricas usando el conjunto de prueba y analice el resultado

## Rúbrica para la Implementación de un Árbol de Decisión

**Nota: Esta rúbrica se basa en la calidad de la implementación y los resultados obtenidos, no en la cantidad de código.**

**1. Creación de la Clase Nodo (10 puntos)**

- [ ] Se crea una clase `Nodo` con los atributos mencionados en las especificaciones (feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha).
- [ ] Los atributos se definen correctamente y se asignan de manera apropiada.

**2. Creación de la Clase Árbol de Decisión (20 puntos)**

- [ ] Se crea una clase que implementa un árbol de decisión.
- [ ] La clase utiliza las funciones presentadas en el cuaderno.
- [ ] Se implementan los hyperparámetros solicitados (max_depth, min_split_samples, criterio).
- [ ] La clase es capaz de entrenar un árbol de decisión con los hyperparámetros especificados.

**3. División de Datos (10 puntos)**

- [ ] Los datos se dividen en conjuntos de entrenamiento y prueba de forma manual.
- [ ] Se utiliza Numpy o Pandas para realizar esta división.
- [ ] Se garantiza que los conjuntos sean excluyentes.

**4. Implementación de Validación Cruzada (20 puntos)**

- [ ] Se implementa la función `validacion_cruzada` correctamente.
- [ ] Los datos de entrenamiento se dividen en k subconjuntos excluyentes.
- [ ] Se entrena y evalúa un modelo para cada subconjunto de validación.
- [ ] Se calculan y reportan las métricas de accuracy, precision, recall y F1.
- [ ] Se calcula la media y la desviación estándar de estas métricas.

**5. Entrenamiento de Modelos (20 puntos)**

- [ ] Se entrenan 10 combinaciones distintas de parámetros para el árbol de decisión.
- [ ] Cada combinación se entrena utilizando la función `validacion_cruzada`.
- [ ] Los resultados de las métricas se registran adecuadamente.

**6. Análisis de Modelos (10 puntos)**

- [ ] Se analizan los resultados obtenidos y se selecciona el mejor modelo para ser utilizado en producción.
- [ ] Se proporciona una justificación clara y fundamentada sobre por qué se eligió ese modelo.

**7. Prueba en el Conjunto de Prueba (10 puntos)**

- [ ] Se comprueban las métricas del modelo seleccionado utilizando el conjunto de prueba.
- [ ] Se analizan los resultados y se comentan las conclusiones.

**General (10 puntos)**

- [ ] El código se documenta de manera adecuada, incluyendo comentarios que expliquen las secciones clave.
- [ ] El código se ejecuta sin errores y sigue buenas prácticas de programación.
- [ ] La presentación de los resultados es clara y fácil de entender.
- [ ] Se cumple con todos los requisitos y las especificaciones proporcionadas.

**Puntuación Total: 100 puntos**

